In [24]:
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import random
import os
#os.chdir('../Chatbot/')
lemmatizer = WordNetLemmatizer()
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = 'intents.json'
with open(data_file) as f:
    intents = json.loads(f.read())


In [25]:
# Perform Tokenization
    
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #adding documents in the corpus
        documents.append((w, intent['tag']))
        
        # adding to classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [37]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), " = number of documents",documents)
# classes = intents
#print (len(classes), "classes = ", classes)
# words = all words, vocabulary
#print (len(words), "unique lemmatized words = ", words)
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))



47  = number of documents [(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'ad

In [27]:
# Lemmatization , lowering each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
#print (len(documents), "number of documents")
#print (len(classes), "classes = ", classes)
#print (len(words), "unique words = ", words)

#Store as a pickle file
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

In [50]:
# Create Training and Testing Data


training = []
# Empty array for output
output_empty = [0]*len(classes)
# training set, bag of words for each sentence
for doc in documents:
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    #print(pattern_words)
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    #print(pattern_words)
for w in words:
    bag.append(1) if w in pattern_words else bag.append(0)
    # output ='0' for each tag and '1' for current tag (for each pattern)
    #print(bag)
    #print(classes)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    #print((doc[1]))
    #print(output_row)
    training.append([bag, output_row])
#print(" next",training )
# shuffle features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
x_train = list(training[:,0])
y_train = list(training[:,1])


In [48]:
# Create a model
model = Sequential()
model.add(Dense(64,input_shape = (len(x_train[0]),),activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(128,activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(len(y_train[0]),activation = 'softmax'))
          
model.compile(optimizer = 'adadelta',loss = 'categorical_crossentropy',metrics = ['accuracy'])
hist = model.fit(np.array(x_train),np.array(y_train),batch_size = 5,epochs = 100)
model.save('Model_sophia.h5',hist)

print(model)

Epoch 1/100
88/88 [==============================] - 0s 2ms/step - loss: 1.3067 - accuracy: 0.9318
Epoch 2/100
88/88 [==============================] - 0s 224us/step - loss: 0.1122 - accuracy: 1.0000
Epoch 3/100
88/88 [==============================] - 0s 220us/step - loss: 0.0236 - accuracy: 1.0000
Epoch 4/100
88/88 [==============================] - 0s 231us/step - loss: 0.0107 - accuracy: 1.0000
Epoch 5/100
88/88 [==============================] - 0s 220us/step - loss: 0.0075 - accuracy: 1.0000
Epoch 6/100
88/88 [==============================] - 0s 216us/step - loss: 0.0032 - accuracy: 1.0000
Epoch 7/100
88/88 [==============================] - 0s 202us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 8/100
88/88 [==============================] - 0s 181us/step - loss: 0.0019 - accuracy: 1.0000
Epoch 9/100
88/88 [==============================] - 0s 172us/step - loss: 0.0014 - accuracy: 1.0000
Epoch 10/100
88/88 [==============================] - 0s 173us/step - loss: 0.0011 - accuracy

88/88 [==============================] - 0s 178us/step - loss: 4.7261e-06 - accuracy: 1.0000
Epoch 79/100
88/88 [==============================] - 0s 174us/step - loss: 3.6264e-06 - accuracy: 1.0000
Epoch 80/100
88/88 [==============================] - 0s 164us/step - loss: 6.1771e-06 - accuracy: 1.0000
Epoch 81/100
88/88 [==============================] - 0s 173us/step - loss: 9.7029e-06 - accuracy: 1.0000
Epoch 82/100
88/88 [==============================] - 0s 164us/step - loss: 7.6885e-06 - accuracy: 1.0000
Epoch 83/100
88/88 [==============================] - 0s 165us/step - loss: 3.5681e-06 - accuracy: 1.0000
Epoch 84/100
88/88 [==============================] - 0s 169us/step - loss: 2.4844e-06 - accuracy: 1.0000
Epoch 85/100
88/88 [==============================] - 0s 168us/step - loss: 3.6927e-06 - accuracy: 1.0000
Epoch 86/100
88/88 [==============================] - 0s 163us/step - loss: 4.5326e-06 - accuracy: 1.0000
Epoch 87/100
88/88 [==============================] - 0s 16